READ ME:

Permissions for this test: 

- No talking until you have left the room.
- This is a closed note, closed book, closed internet exam.
- You may have one jupyter notebook (this notebook) open for the duration of the exam.
- You may have one tab open solely for the act of submitting your exam.

To  begin the exam:

- **Rename the notebook to be of the form `<uni>_exam`. For example, mine would be `pl2648_exam`.**

When you are done with your exam:

1. Save this exam.
1. Download this exam from your Jupyter Notebook tab as an **`.ipynb`** file.
1. Upload/email/etc the **`.ipynb`** file to the submission platform designated by the exam proctor.

----

Please note, there are several cells in this Jupyter notebook that are empty and read only. Do not attempt to remove them or edit them. They are used in grading your notebook.

- DO remove the "Not Implemented" lines if you at all attempt the problem.
- DO test all cells to make sure they run in 30 seconds or less.

### Question

Write a class named `StopWatch`. This class should have `start`, `stop`, and `lap` methods. 

An instance of the `StopWatch` class should keep track of each start-stop pair that have occurred; much like an exercise stop watch. After `start` is called, `stop` can be called to track the time since `start` was called. If `start` has been called the only valid method call is `stop`. If `stop` has been called, `start` or `lap` can be called. `stop` can also be called again but it should have no effect on the state of the stop watch. If `start` has been called but `stop` has not yet been called and `start` or `lap` are called, a `RuntimeError` should be raised. 

`lap` must be called with a zero based integer index into the sequence of start-stop pairs. `lap` should return the time between the start and stop time stamps for that lap. If an index passed to `lap` is out of bounds of the valid laps, an `IndexError` should be raised. The units of time can be equivalent to those provided by the standard library's `time.time` function.


```python
sw = StopWatch()
sw.start()
# wait about five seconds
sw.stop()
sw.lap(0)
    5.007094860076904

sw.lap(1)
    IndexError: list index out of range

sw.start()
sw.start()
    RuntimeError: Can not start StopWatch when it is already running
```
[4 points]

In [47]:
### BEGIN SOLUTION
import time

class StopWatch:
    def __init__(self):
        self.laps = []
        # If _start is not None, we can assume the StopWatch is running.
        self._start = None
    
    def start(self):
        if self._start is not None:
            raise RuntimeError('Can not start StopWatch when it is already running')
        self._start = time.time()
    
    def stop(self):
        # Let's not concern ourselves if stop was called twice
        if self._start is None:
            return
        
        # Record lap
        self.laps.append((
            self._start,
            time.time()
        ))
        
        # Stop stopwatch
        self._start = None
    
    def lap(self, index):
        start, stop = self.laps[index]
        return stop - start
### END SOLUTION

In [48]:
### BEGIN HIDDEN TESTS
import time

sw = StopWatch()
sw.start()
time.sleep(0.5)
sw.stop()
interval = sw.lap(0)
assert 0.4 < interval < 0.6, interval
### END HIDDEN TESTS

In [49]:
### BEGIN HIDDEN TESTS
sw = StopWatch()
sw.start()
try:
    sw.start()
    assert False, 'Should have caused a RuntimeError'
except RuntimeError:
    pass
### END HIDDEN TESTS

In [50]:
### BEGIN HIDDEN TESTS
import time

sw = StopWatch()
for i in range(1, 10):
    sw.start()
    time.sleep(1 / i)
    sw.stop()
### END HIDDEN TESTS

In [51]:
### BEGIN HIDDEN TESTS
assert sw.lap(5)
### END HIDDEN TESTS

In [52]:
### BEGIN HIDDEN TESTS
try:
    sw.lap(11)
    assert False, 'Should have caused IndexError'
except IndexError:
    pass
### END HIDDEN TESTS

### Question 

Create a function named `iterable_to_dict`. This function should take any iterable and add its members to a dictionary as members of lists where the member containing lists are the values of the dictionary. The key for each value (each list) should be the first list item's address in memory expressed as a hex address string. If a value for the optional keyword argument `key_func` is passed to the function, the `key_func` should be used to determine an item's key. Each item should be passed to `key_func` to determine the list's key. If there is a key collision, the list of items for a particular key should be increased to include the new item.

For example:

```python
iterable_to_dict((1, 2, 3, 4, 5))
{'0x109258440': [1],
 '0x109258460': [2],
 '0x109258480': [3],
 '0x1092584a0': [4],
 '0x1092584c0': [5]}


iterable_to_dict((1, 2, 3, 4, 5), key_func=lambda x: 'a')
{'a': [1, 2, 3, 4, 5]}
```

[3 points]

In [2]:
def key_func(value):
    return bin(value)

iterable_to_dict(iterable, key_func=key_func)

'0b1'

In [1]:
### BEGIN SOLUTION
def iterable_to_dict(iterable, key_func=None):
    result = {}
    for item in iterable:
        if key_func:
            key = key_func(item)
        else:
            key = hex(id(item))
        
        if key in result:
            result[key].append(item)
        else:
            result[key] = [item]
    
    return result
### END SOLUTION

In [2]:
### BEGIN HIDDEN TESTS
i = (1, 2, 3, 4, 5)
d = iterable_to_dict(i)
assert all(k.startswith('0x') for k in d)
### END HIDDEN TESTS

In [24]:
### BEGIN HIDDEN TESTS
i = (1, 2, 3, 4, 5)
d = iterable_to_dict(i)
assert sorted(list(d.values())) == [[1], [2], [3], [4], [5]]
### END HIDDEN TESTS

In [28]:
### BEGIN HIDDEN TESTS
i = (1, 2, 3, 4, 5)
d = iterable_to_dict(i, key_func=lambda x: x % 2)
assert d == {1: [1, 3, 5], 0: [2, 4]}
### END HIDDEN TESTS

### Question

NYC has all sorts of taxi types. Yellow Cab, Service Car, Uber, Lyft, maybe even 滴滴.

Let's assume each license plate number can give us hints on what service the car drives for. For example, if a license plate number starts with a T, we can assume its a taxi of some sort. If the number ends with a C, then it is a Yellow Cab. If it ends with a U, then the taxi is an Uber. The rest of the mappings are as described below.

- T123E5C           # Yellow Cab
- T34567S           # Service Car
- T5J788U           # Uber
- T987Q5L           # Lyft
- T27865D           # DiDi 
- 36A78ED           # Not a taxi

The number/letter comboniation in between the first and last digits give us the taxi ID if the plate number is for a taxi. Otherwise, they are just part of the greater license plate number.

Write a regular expression to return an `re.match` object with the following match groups if compared with a taxi number. Otherwise, the pattern should cause `re.match` to return `None`. You should assume the taxi ID will always be 5 digits and the service type will always be 1 digit long. Please note, you are being asked to creat

```python
"T<taxi_id><service_type>"
```

[3 points]

In [34]:
import re

def parse_number(plate_number):
    ### BEGIN SOLUTION
    pattern_string = r"^T(?P<taxi_id>\w{5})(?P<service_id>\w{1})"
    ### END SOLUTION
    return re.match(pattern_string, plate_number)

In [35]:
### BEGIN HIDDEN TESTS
match = parse_number('T123E5C')
taxi_id = match.group(1)
assert taxi_id == '123E5', taxi_id
### END HIDDEN TESTS

In [36]:
### BEGIN HIDDEN TESTS
match = parse_number('T123E5C')
service_id = match.group(2)
assert service_id == 'C', service_id
### END HIDDEN TESTS

In [37]:
### BEGIN HIDDEN TESTS
match = parse_number('T34567S')
taxi_id = match.group(1)
assert taxi_id == '34567', taxi_id
### END HIDDEN TESTS

In [38]:
### BEGIN HIDDEN TESTS
match = parse_number('T34567S')
service_id = match.group(2)
assert service_id == 'S', service_id
### END HIDDEN TESTS

In [39]:
### BEGIN HIDDEN TESTS
match = parse_number('T5J788U')
taxi_id = match.group(1)
assert taxi_id == '5J788', taxi_id
### END HIDDEN TESTS

In [40]:
### BEGIN HIDDEN TESTS
match = parse_number('T5J788U')
service_id = match.group(2)
assert service_id == 'U', service_id
### END HIDDEN TESTS

In [41]:
### BEGIN HIDDEN TESTS
match = parse_number('T987Q5L')
taxi_id = match.group(1)
assert taxi_id == '987Q5', taxi_id
### END HIDDEN TESTS

In [42]:
### BEGIN HIDDEN TESTS
match = parse_number('T987Q5L')
service_id = match.group(2)
assert service_id == 'L', service_id
### END HIDDEN TESTS

In [43]:
### BEGIN HIDDEN TESTS
match = parse_number('T27865D')
taxi_id = match.group(1)
assert taxi_id == '27865', taxi_id
### END HIDDEN TESTS

In [44]:
### BEGIN HIDDEN TESTS
match = parse_number('T27865D')
service_id = match.group(2)
assert service_id == 'D', service_id
### END HIDDEN TESTS

In [45]:
### BEGIN HIDDEN TESTS
match = parse_number('36A78ED')
assert match is None, match
### END HIDDEN TESTS

### Question

You are given a nested data structure like the following:

```python
{
    'a': {
        'b': 'c', 
        'd': 'e',
        'f': ['g', 'h', 'i'],
    },
    'j': ('k', 'l', 'm'),
    'n': 'o',
    'p': {'q', 'r', 's', 't'},
    'u': {
        'v': {
            'w': 'x',
            'y': 'z'
        }
    }
}
```

Write a recursive function `count_values` to find the number of values (for all keys) in the structure. However, the number of values should be counted in a special way. 

- If the value is a string, count the number of characters in the string.
- If the value is a list, tuple, or set, recurisvely count the number of items in the iterable. Be careful to not count the list, tuple, or set itself.
- If the value is a dict, recursively count the number of values in the dict.

In the example above, the result should be 15.

You can not assume the outermost structure is a dictionary. Your function should take one positional argument.

[3 points]

In [22]:
def count_values(obj):
    ### BEGIN SOLUTION
    
    count = 0

    if isinstance(obj, (str,)):
        count = len(obj)

    elif isinstance(obj, (list, tuple, set)):
        for item in obj:
            count += count_values(item)

    elif isinstance(obj, dict):
        for value in obj.values():
            count += count_values(value)
            
    return count  
    ### END SOLUTION

In [36]:
### BEGIN HIDDEN TESTS
data1 = {
    'a': {
        'b': 'c', 
        'd': 'e',
        'f': ['g', 'h', 'i'],
    },
    'j': ('k', 'l', 'm'),
    'n': 'o',
    'p': {'q', 'r', 's', 't'},
    'u': {
        'v': {
            'w': 'x',
            'y': 'z'
        }
    }
}

data2 = [
    {
        'a': 'b',
        'c': 'd',
        'e': ['fghi'],
    },
    {
        'j': {'klmn'},
    }
]
### END HIDDEN TESTS

In [30]:
### BEGIN HIDDEN TESTS
assert count_values(data1) == 15
### END HIDDEN TESTS

In [31]:
### BEGIN HIDDEN TESTS
assert count_values(data1['a']) == 5
### END HIDDEN TESTS

In [32]:
### BEGIN HIDDEN TESTS
assert count_values(data1['u']) == 2
### END HIDDEN TESTS

In [33]:
### BEGIN HIDDEN TESTS
assert count_values(data1['a']['b']) == 1
### END HIDDEN TESTS

In [38]:
### BEGIN HIDDEN TESTS
assert count_values(data2) == 10
### END HIDDEN TESTS

In [41]:
### BEGIN HIDDEN TESTS
assert count_values({}) == 0
### END HIDDEN TESTS